In [ ]:
import sys
import os

# Get the path of the current notebook's directory and append the parent directory to the path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import importlib

import pdc_functions.assumption_utils as assumptions
import pdc_functions.data_utils as data
import pdc_functions.explore_utils as explore
import pdc_functions.pdc_1and2_utils as pdc
import pdc_functions.stats_utils as stats
from dotenv import load_dotenv
import pdc_functions.class_utils as class_utils
import pandas as pd


from phmlondon.snow_utils import SnowflakeConnection


FileNotFoundError: [Errno 2] No such file or directory

In [105]:
importlib.reload(stats)

<module 'pdc_functions.stats_utils' from '/Users/jsaund/Documents/repos/onelondon_snowflake_datascience/projects/meds_compliance/pdc_functions/stats_utils.py'>

In [ ]:
load_dotenv()

snowsesh = SnowflakeConnection()
snowsesh.use_database("INTELLIGENCE_DEV")
snowsesh.use_schema("AI_CENTRE_FEATURE_STORE")

In [ ]:
ldl = data.get_data_by_cohort(snowsesh, "intelligence_dev.ai_centre_dev.ldl_all")


Clean the ldl column first

In [ ]:
print(ldl)

In [ ]:
statin = data.get_data_by_cohort(snowsesh, "intelligence_dev.ai_centre_dev.statin_all")

In [ ]:
drug_table = class_utils.MedicationTable(statin)


In [ ]:
drug_table.validate_columns()  # Validate columns
drug_table.normalise_missing_values()  # Standardise representation of missing data
drug_table.convert_dates()  # Convert date columns
drug_table.validate_data_types()  # Validate data types
drug_table.clean_dose()  # Clean and interpret the 'dose' column
drug_table.calculate_covered_days()

print("All validations passed successfully!")

In [ ]:
person_drug_pdc = pdc.compute_pdc_overall(drug_table.df)

In [ ]:
person_drug_pdc = data.add_demographic_data(snowsesh, person_drug_pdc)

In [ ]:
person_drug_pdc = data.general_agg(person_drug_pdc)

In [ ]:
print(person_drug_pdc)

Join to results table

In [ ]:
data_joined = data.attach_closest_results(person_drug_pdc,ldl)

In [ ]:
print(data_joined)


In [ ]:
data_joined_2 =  data.attach_closest_results_2(person_drug_pdc,ldl)

Clean 
- remove people without at least a year of medication
- remove people with missing bloods - need at least 2 before and 2 after for average


In [ ]:
clean_data = data.cohort_exclusions(data_joined)

In [96]:
print(clean_data)

        person_id                  drug_name min_start_date max_start_date  \
5              10                Simvastatin     2008-06-18     2015-02-26   
7              14                Simvastatin     2009-10-28     2013-07-01   
10             16                Simvastatin     2007-12-14     2016-08-17   
11             18                Simvastatin     2002-08-28     2011-10-18   
14             24               Atorvastatin     2002-04-23     2006-10-03   
...           ...                        ...            ...            ...   
415798    6403756               Atorvastatin     2014-09-22     2020-08-25   
415799    6403756                Simvastatin     2012-09-13     2014-11-11   
415805    6412440                Simvastatin     2011-02-15     2012-12-11   
415812    6433901  Omega-3-acid ethyl esters     2014-09-24     2020-04-22   
415816    6482170                Simvastatin     2005-04-15     2010-05-06   

        gender    ethnicity  imd  age_at_start  overall_inclusi

Average the results before/after

In [102]:
analysis_data = data.avg_results(clean_data)
print(analysis_data)

        person_id                  drug_name min_start_date max_start_date  \
5              10                Simvastatin     2008-06-18     2015-02-26   
7              14                Simvastatin     2009-10-28     2013-07-01   
10             16                Simvastatin     2007-12-14     2016-08-17   
11             18                Simvastatin     2002-08-28     2011-10-18   
14             24               Atorvastatin     2002-04-23     2006-10-03   
...           ...                        ...            ...            ...   
415798    6403756               Atorvastatin     2014-09-22     2020-08-25   
415799    6403756                Simvastatin     2012-09-13     2014-11-11   
415805    6412440                Simvastatin     2011-02-15     2012-12-11   
415812    6433901  Omega-3-acid ethyl esters     2014-09-24     2020-04-22   
415816    6482170                Simvastatin     2005-04-15     2010-05-06   

        gender    ethnicity  imd  age_at_start  overall_inclusi

In [ ]:
# Unadjusted Linear regression

In [106]:
linr_unadj = stats.unadjusted_linr(analysis_data, "overall_exclusive_pdc", "result_diff")

In [107]:
print(linr_unadj.summary())

                            OLS Regression Results                            
Dep. Variable:            result_diff   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     388.6
Date:                Sat, 10 May 2025   Prob (F-statistic):           2.27e-86
Time:                        12:29:23   Log-Likelihood:            -2.7051e+05
No. Observations:              126581   AIC:                         5.410e+05
Df Residuals:                  126579   BIC:                         5.410e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.72